In [ ]:
import pandas as pd
df = pd.read_csv(r"C:\Users\yaoyan\Downloads\bquxjob_4e43d5f5_1915b006544.csv")

In [ ]:
df

In [ ]:
df_melted = pd.melt(df, 
                    id_vars=['platform_id', 'user_id', 'omg_play_cnt'], 
                    value_vars=[col for col in df.columns if col.startswith('rtp_')],
                    var_name='局数', 
                    value_name='值')

# 修改 '局数' 列的值
df_melted['局数'] = df_melted['局数'].str.replace('rtp_', '')

mapping = {
    '0_50': 'a.0_50', '50_100': 'b.50_100', '100_150': 'c.100_150', '150_200': 'd.150_200',
    '200_250': 'e.200_250', '250_300': 'f.250_300', '300_350': 'g.300_350', '350_400': 'h.350_400',
    '400_450': 'i.400_450', '450_500': 'j.450_500', '500_': 'k.500_'
}

df_melted['局数'] = df_melted['局数'].map(mapping)
df_melted.to_excel(r"C:\Users\yaoyan\Downloads\rtp波动逆向.xlsx")


In [ ]:
!pip install dingtalkchatbot

### 上海数据播放用 钉钉机器人

In [1]:
from dingtalkchatbot.chatbot import DingtalkChatbot
def postDing(text, at_mobiles=[19901942239], is_at_all=False):
    webhook = r"https://oapi.dingtalk.com/robot/send?access_token=32b80af90566ea0a4d335ead1d851ac384f64d67ca01f35965d152b81e6f0f30"
    xiaoding = DingtalkChatbot(webhook)  # Ding Ding object
    response = xiaoding.send_text(msg=text + '.', is_at_all=is_at_all, at_mobiles=at_mobiles)

In [2]:
import pandas as pd
df = pd.read_excel(r"C:\Users\yaoyan\Downloads\上海播放数据用.xlsx",sheet_name = '整体表',usecols = list(range(0,48)))

import time,datetime
import pytz
china_tz = pytz.timezone('Asia/Shanghai')
china_now = datetime.datetime.now(china_tz)

# 若日期不符 此处更正
delta = datetime.timedelta(days=1)
china_now = china_now - delta

china_month =china_now.month 
china_yes = china_now - datetime.timedelta(days = 1)
china_yes_day = china_yes.strftime('%Y-%m-%d')
days = china_yes.day # 算日均用 本月过去多少天

# 常规情况播放
text1 = f'''
上海铁三角数据播放，截止{china_yes_day}
part1 整体情况
1、规模情况：
a.当月新增发展{df['current_month_dg_cnt'].values[0]}，日均{round(df['current_month_dg_cnt'].values[0]/days)}
b.当月净增规模{df['current_month_settle_cnt'].values[0]-df['last_month_end_settle_cnt'].values[0]}
2、代销发展情况：
a.当月新增发展{df['current_month_agent_dgcnt'].values[0]}，日均{round(df['current_month_agent_dgcnt'].values[0]/days)}
b.当月发展首月退订率{round(df['current_month_agent_tdcnt'].values[0]/df['current_month_agent_dgcnt'].values[0]*100,2)},环比上月{round(round(df['current_month_agent_tdcnt'].values[0]/df['current_month_agent_dgcnt'].values[0]*100,2) - round(df['last_month_agent_tdcnt'].values[0]/df['last_month_agent_dgcnt'].values[0]*100,2),2)}pp
c.当月退订{df['current_month_agent_unsub_mon_cnt'].values[0]}，日均{round(df['current_month_agent_unsub_mon_cnt'].values[0]/days)}
d.整体发展整体退订数{df['current_month_agent_total_tdcnt'].values[0]}，累计代销退订率{round(df['current_month_agent_total_tdcnt'].values[0]/df['current_month_agent_total_dgcnt'].values[0]*100,2)}%
e.代销占比整体规模情况{round(df['current_month_agent_dgcnt'].values[0]/df['current_month_dg_cnt'].values[0] * 100 ,2)}%，环比{round(round(df['current_month_agent_dgcnt'].values[0]/df['current_month_dg_cnt'].values[0] * 100 ,2)-round(df['last_month_agent_dgcnt'].values[0]/df['last_month_dg_cnt'].values[0] * 100 ,2),2)}pp
3、结算收入情况
a.当月整体结算量{df['current_month_settle_cnt'].values[0]}，环比{round( (df['current_month_settle_cnt'].values[0]-df['last_month_settle_cnt'].values[0])/df['last_month_settle_cnt'].values[0] * 100 ,2 )}%
b.当月整体收入含{df['current_month_pft_cmsn_amt'].values[0]}，环比{round( (df['current_month_pft_cmsn_amt'].values[0]-df['last_month_pft_cmsn_amt'].values[0])/df['last_month_pft_cmsn_amt'].values[0]  * 100 ,2 )}%
c.当月整体单包毛利含{round(df['current_month_pft_cmsn_amt'].values[0]/df['current_month_settle_cnt'].values[0],2)}，环比{round( ( df['current_month_pft_cmsn_amt'].values[0]/df['current_month_settle_cnt'].values[0]- df['last_month_pft_cmsn_amt'].values[0]/df['last_month_settle_cnt'].values[0])/ (df['last_month_pft_cmsn_amt'].values[0]/df['last_month_settle_cnt'].values[0]) * 100 ,2)}%
'''

print(text1)
# 播放  1
# postDing(text1)


# 利润情况预测
上月底结算量 = df['last_month_end_settle_cnt'].values[0]
本月留存用户 = df['current_month_old_settle_cnt'].values[0]
留存率 = round(本月留存用户/上月底结算量 - 0.04,2) # 平滑数值
本月结算用户 = df['current_month_settle_cnt'].values[0]

本月单包毛利 = int(round(df['current_month_pft_cmsn_amt'] /df['current_month_settle_cnt'],2))

# part1 存量利润获取
存量利润 = 0
for i in range(china_month-china_month+1,13-china_month+1):
    # print(i)
    存量每月利润 = 留存率 **i * 本月结算用户 *  本月单包毛利
    # print(留存率 **i * 本月结算用户)
    # print(存量每月利润)
    存量利润 = 存量利润 +存量每月利润
print('存量利润为：',存量利润)
    
# part2 新增利润获取
df = pd.read_excel(r"C:\Users\yaoyan\Downloads\上海播放数据用.xlsx",sheet_name = '当月新增代销',usecols = [0,1,2])
df = df.rename(columns = {'current_month_agent_dgcnt':'结算量'})
df = df.drop(df[df['sales_nm'] == '上海购物卡'].index)
df = df.drop(df[df['结算量'] == 0].index)
df['占比'] = round(df['结算量']/df['结算量'].sum() * 100 ,2 )
df['占比'] = df['占比'].astype(str) + '%'

单包毛利字典 = {'SP202209161809376608974':6.5,'SP202209161858279497030':9}
白金日均 = df[df['sales_id'] == 'SP202209161809376608974']['结算量'].values[0]/days
黑金日均 = df[df['sales_id'] == 'SP202209161858279497030']['结算量'].values[0]/days

代销留存率 = 0.68
新增利润 = 0
for i in range(china_month-china_month,13-china_month):
    # print(i)
    白金利润 = (代销留存率 ** i) * (白金日均 *30) * 单包毛利字典['SP202209161809376608974']
    黑金利润 = (代销留存率 ** i) * (黑金日均 *30) * 单包毛利字典['SP202209161858279497030']
    # print(白金利润+黑金利润)
    新增利润 = 新增利润+白金利润+黑金利润

print('新增利润为：',新增利润)


# part3 后续月份新增计算
日均 = 1500
天数 = 30
单包利润 = (0.5*9+0.5*6.5) # 按照
剩余月份长度 = len(list(range(china_month-china_month,13-china_month)))-1
循环月份数 = list(range(china_month-china_month,13-china_month-1))
后续月份利润 = 0
for i in range(剩余月份长度,0,-1):
    当月循环月份数 = 循环月份数[:i]
    # print(当月循环月份数)
    for i in 当月循环月份数:
        后续月份利润 = 后续月份利润 + (代销留存率 ** i)*日均*天数*单包利润

print('后续月份利润为：',后续月份利润)

# part4 制造合计
月份1利润 = 490397
月份2利润 = 495263
月份3利润 = 477337
月份4利润 = 555576
月份5利润 = 763106
月份6利润 = 775982
月份7利润 = 874372
截止7月份利润 = 月份1利润+月份2利润+月份3利润+月份4利润+月份5利润+月份6利润+月份7利润
总利润 = 截止7月份利润+存量利润+新增利润+后续月份利润

print('总利润为：',总利润)
预测指标完成度= round(总利润/11012600 * 100 ,2)
print('预测指标完成度：',预测指标完成度)



text2 = f'''
part2 收入指标完成预测
截止7月份利润为：{截止7月份利润}
8月存量部分预测为：{存量利润}
8月新增利润预测为：{新增利润}
9月与之后月份利润为：{后续月份利润}
总利润为：{总利润}
预计指标完成度{预测指标完成度}%
'''

print(text2)
# 播放  2
# postDing(text2)


上海铁三角数据播放，截止2024-08-31
part1 整体情况
1、规模情况：
a.当月新增发展69253，日均2234
b.当月净增规模9121
2、代销发展情况：
a.当月新增发展56291，日均1816
b.当月发展首月退订率30.93,环比上月4.96pp
c.当月退订36316，日均1171
d.整体发展整体退订数119429，累计代销退订率50.76%
e.代销占比整体规模情况81.28%，环比-0.12pp
3、结算收入情况
a.当月整体结算量281063，环比3.35%
b.当月整体收入含976324.0025，环比11.66%
c.当月整体单包毛利含3.47，环比8.04%

存量利润为： 1680214.08604464
新增利润为： 598722.033135484
后续月份利润为： 1681812.0
总利润为： 8392781.119180124
预测指标完成度： 76.21

part2 收入指标完成预测
截止7月份利润为：4432033
8月存量部分预测为：1680214.08604464
8月新增利润预测为：598722.033135484
9月与之后月份利润为：1681812.0
总利润为：8392781.119180124
预计指标完成度76.21%



C:\Users\yaoyan\AppData\Local\Temp\ipykernel_10132\1334962904.py:2: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel(r"C:\Users\yaoyan\Downloads\上海播放数据用.xlsx",sheet_name = '整体表',usecols = list(range(0,48)))


In [3]:
import pandas as pd
df = pd.read_excel(r"C:\Users\yaoyan\Downloads\上海播放数据用.xlsx",sheet_name = '当月新增代销',usecols = [0,1,2])
df = df.rename(columns = {'current_month_agent_dgcnt':'结算量'})
df = df.drop(df[df['sales_nm'] == '上海购物卡'].index)
df = df.drop(df[df['结算量'] == 0].index)
df['占比'] = round(df['结算量']/df['结算量'].sum() * 100 ,2 )
df['占比'] = df['占比'].astype(str) + '%'
df = df.loc[:,'sales_nm':'占比'].reset_index(drop = True)
text = '''
'''
text_list = []
for key,row in df.iterrows():
    text_list.append([str(row['sales_nm']),'结算量：'+str(row['结算量']),'占比：'+str(row['占比'])])
for _ in text_list:
    text = text + ','.join(_)+'\n'

text3 = 'part3 分销售品看代销情况'+ text  + '黑金/白金：'+ str(round(df[df['sales_nm'] == '橙翼会员黑金版（12期）']['结算量'].values[0]/df[df['sales_nm'] == '橙翼会员白金版（12期）']['结算量'].values[0] ,2))
print(text3)

part3 分销售品看代销情况
橙翼会员黑金版（12期）,结算量：11717,占比：34.18%
省钱流量小合约,结算量：49,占比：0.14%
橙翼会员白金版（12期）,结算量：22518,占比：65.68%
黑金/白金：0.52


In [4]:
text = text1+text2+'\n'+text3
print(text)


上海铁三角数据播放，截止2024-08-31
part1 整体情况
1、规模情况：
a.当月新增发展69253，日均2234
b.当月净增规模9121
2、代销发展情况：
a.当月新增发展56291，日均1816
b.当月发展首月退订率30.93,环比上月4.96pp
c.当月退订36316，日均1171
d.整体发展整体退订数119429，累计代销退订率50.76%
e.代销占比整体规模情况81.28%，环比-0.12pp
3、结算收入情况
a.当月整体结算量281063，环比3.35%
b.当月整体收入含976324.0025，环比11.66%
c.当月整体单包毛利含3.47，环比8.04%

part2 收入指标完成预测
截止7月份利润为：4432033
8月存量部分预测为：1680214.08604464
8月新增利润预测为：598722.033135484
9月与之后月份利润为：1681812.0
总利润为：8392781.119180124
预计指标完成度76.21%

part3 分销售品看代销情况
橙翼会员黑金版（12期）,结算量：11717,占比：34.18%
省钱流量小合约,结算量：49,占比：0.14%
橙翼会员白金版（12期）,结算量：22518,占比：65.68%
黑金/白金：0.52


In [5]:
postDing(text)